In [9]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from preprocessing.dataprep import data_preparation, filter_tce_data, encode_train_test
from preprocessing.text import fixColumnName
from sklearn.model_selection import train_test_split


In [10]:
data = pd.read_csv('../database/dadosTCE.csv', low_memory=False)
data = data.sample(1000,).reset_index(drop=True)

In [11]:
data.columns = list(map(fixColumnName, data.columns))
data, _ = filter_tce_data(data, '../database/norel.xlsx')

categorical_columns = [
    # 'exercicio_do_orcamento_ano',
    # 'empenho_sequencial_empenho',
    'orgao',
    'orgao_sucessor_atual',
    'tipo_administracao_nome',
    'tipo_poder_nome',
    # 'classificacao_orcamentaria_descricao',
    'funcao',
    'subfuncao',
    'programa',
    'acao',
    'grupo_despesa',
    'elemento_despesa',
    # 'natureza_despesa_cod',
    # 'natureza_despesa_nome',
    # 'formalidade_nome',
    'modalidade_licitacao_nome',
    # 'fonte_recurso_cod',
    'fonte_recurso_nome',
    'beneficiario_cnpj',
    'beneficiario_cpf',
    'beneficiario_cpf/cnpj',
    # 'periodo',
    'empenho_numero_do_processo',
    # 'empenho_sequencial_empenho.1',
]

text_columns = [
    # 'beneficiario_nome',
    'empenho_historico',
]

numerical_columns = [
    # 'valor_empenhado',
    # 'valor_anulacao_empenho',
    # # 'valor_estorno_anulacao_empenho',
    # 'valor_cancelamento_empenho',
    # # 'valor_anulacao_cancelamento_empenho',
    # 'valor_saldo_do_empenho',
    # 'valor_liquidacao_empenho',
    # 'valor_anulacao_liquidacao_empenho',
    # 'valor_saldo_liquidado',
    # 'valor_ordem_de_pagamento',
    # 'valor_guia_recolhimento',
    # 'valor_anulacao_ordem_de_pagamento',
    # 'valor_estorno_anulacao_o._pagamento',
    # 'valor_estorno_guia_recolhimento',
    # 'valor_saldo_pago',
    # 'valor_saldo_a_pagar',
    # 'valor_a_liquidar',
    # 'valor_a_pagar_liquidado'
]

target = data.valor_empenhado

data = data.loc[:, (*categorical_columns,
                    *text_columns,
                    *numerical_columns,)]

data, categorical_columns, numerical_columns = data_preparation(
    data,
    target,
    test_size=0.3,
    categorical_columns=categorical_columns,
    numerical_columns=numerical_columns,
    text_columns=text_columns,)

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.3, random_state=15)

X_train, X_test = encode_train_test(
    X_train, X_test, numerical_columns, categorical_columns, text_columns, tfidf=True,)


In [12]:
reg = SVR(kernel='linear', C=10)
reg.fit(X_train, y_train)

SVR(C=10, kernel='linear')

In [15]:
y_pred = reg.predict(X_test)

In [21]:
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

explained_variance_score(y_test, y_pred), r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred)


(0.0001768486321434004, -0.06243990726848314, 3058217367003.1367)

In [ ]:
reg = RandomForestRegressor(n_estimators=1500, random_state=15)
reg.fit(X_train, y_train)

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
explained_variance_score(y_test, y_pred), r2_score(
    y_test, y_pred), mean_squared_error(y_test, y_pred)
